In [1]:
import os
import subprocess

This notebook is to compare the results obtained with code_refactor branch to the ones obtained with the privious version. I have run the following nexflow command:

```
nextflow run main.nf -resume -profile crg_hpc \
    --tsv /users/rg/fzanarello/test/geneidx/test_data/input_species.tsv \
    --column_taxid_value TAXID \
    --column_path_value PATH \
    --column_id_value ID \
    --outdir /users/rg/fzanarello/test/geneidx/test_data/ \
    --use_masking false
```

In [11]:
# input file
! cat /users/rg/fzanarello/test/geneidx/test_data/input_species.tsv

ID	PATH	TAXID
Arabidopsis_thaliana	https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-57/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna_rm.toplevel.fa.gz	3702
Betta_splendens	/users/rg/fzanarello/test/geneidx/test_data/Betta_splendens.fBetSpl5.2.dna_rm.toplevel.fa.gz	158456


This was to test the possibility of running GeneIdX from a reference genome from url.  
The comparison below only referes to Betta splendens as no previous run for Arabidopsis are available. 

### To extract protein seq from fasta and annotation 

In [2]:
# will use this script to run agtc command 
! cat ../scripts/get_protein_from_gff.sh

#!/bin/bash

#check for input arguments
 
function check_args() {
  if [ $# -ne 3 ]; then
    echo "Error: Exactly 3 arguments required!"
    return 1
  fi
  
  echo "gff: $1"
  echo "reference: $2"
  echo "output dir: $3"
}
 
check_args $1 $2 $3

file_name=$(basename -s .gff3 $1)
file_path=$(echo ${3}${file_name}.protein.fa)


# make result folder if not already exists
mkdir -p $3

echo 'Result file will be saved as: '$file_path


# run agtc singularity 
singularity run /nfs/users/rg/fzanarello/images/agat-1.2.0--pl5321hdfd78af_0.simg agat_sp_extract_sequences.pl -f $2 -g $1 -t cds -p -o $file_path


In [10]:
! ls /nfs/users/rg/fzanarello/projects/busco_benchmark/genomes/Betta*

/nfs/users/rg/fzanarello/projects/busco_benchmark/genomes/Betta_splendens.fBetSpl5.2.dna_rm.toplevel.fa
/nfs/users/rg/fzanarello/projects/busco_benchmark/genomes/Betta_splendens.fBetSpl5.2.dna_rm.toplevel.fa.index


In [13]:
gff = '/users/rg/fzanarello/test/geneidx/output/species/158456/Betta_splendens.gff3'
ref = '/nfs/users/rg/fzanarello/projects/busco_benchmark/genomes/Betta_splendens.fBetSpl5.2.dna_rm.toplevel.fa'
out_folder = '/users/rg/fzanarello/test/geneidx/output/species/158456/proteins'

with open ('../array_commands/extract_protein_coderef_test.txt', 'w') as out:
    out.write(f'/nfs/users/rg/fzanarello/projects/busco_benchmark/scripts/get_protein_from_gff.sh {gff} {ref} {out_folder} \n')  

In [14]:
cat ../array_commands/extract_protein_coderef_test.txt

/nfs/users/rg/fzanarello/projects/busco_benchmark/scripts/get_protein_from_gff.sh /users/rg/fzanarello/test/geneidx/output/species/158456/Betta_splendens.gff3 /nfs/users/rg/fzanarello/projects/busco_benchmark/genomes/Betta_splendens.fBetSpl5.2.dna_rm.toplevel.fa /users/rg/fzanarello/test/geneidx/output/species/158456/proteins 


### To Run BUSCO

In [1]:
! cat ../scripts/run_busco_from_protein.sh

#!/bin/bash

#check for input arguments
 
function check_args() {
  if [ $# -ne 3 ]; then
    echo "Error: Exactly 3 arguments required!"
    return 1
  fi
  
  echo "protein_sequence: $1"
  echo "ortho_db: $2"
  echo "output dir: $3"
}
 
check_args $1 $2 $3


singularity run /nfs/users/rg/fzanarello/images/busco-v5.5.0_cv1.simg busco -m protein -i $1 -l $2 -o $3

In [19]:
annotation = '/users/rg/fzanarello/test/geneidx/output/species/158456/proteinsBetta_splendens.protein.fa'
ortho_db = 'vertebrata_odb10'
out_folder = '/users/rg/fzanarello/test/geneidx/output/species/158456/busco'

with open ('../array_commands/busco_from_protein_refcode_test.txt', 'w') as out:
    out.write(f'/nfs/users/rg/fzanarello/projects/busco_benchmark/scripts/run_busco_from_protein.sh {annotation} {ortho_db} {out_folder} \n')

In [20]:
! cat ../array_commands/busco_from_protein_refcode_test.txt

/nfs/users/rg/fzanarello/projects/busco_benchmark/scripts/run_busco_from_protein.sh /users/rg/fzanarello/test/geneidx/output/species/158456/proteinsBetta_splendens.protein.fa vertebrata_odb10 /users/rg/fzanarello/test/geneidx/output/species/158456/busco 
